In [121]:
import pandas as pd
import numpy as np
#data_path = "C:/Users/lukec/OneDrive - University of Cambridge/PhD/Data/"
data_path = "C:/Users/lshc3/Documents/"
class LCA(object):
    def __init__(self, data_path):
        self.data = pd.DataFrame()
        lca_lists = ['Basic_chemicals_201','Coke_Petro_19']
        for path in lca_lists:
            filepath = data_path+"EcoInvent/"+path+"/GWP100a_IPCC2013.csv"
            raw = pd.read_csv(filepath)
            self.data = self.data.append(raw[['name','location','CO2e']][~raw['generalComment'].str.contains('market')].sort_values('name'))

    def location(self, location='Global'):
        loc_filepath = data_path+"EcoInvent\EcoInvent_locations.csv"
        loc_codes = pd.read_csv(loc_filepath)
        region_code = loc_codes['Code'][np.where(loc_codes['Name'] == location)[0]].iloc[0]
        rows = []
        for product in self.data['name'].unique():
            for code in [region_code, 'RoW', 'GLO']:
                found = self.data[(self.data['name']==product) & (self.data['location']==code)]
                if not found.index.empty:
                    rows += list(found.index)
                    break
        return self.data.loc[rows].groupby(['name','location']).mean().reset_index()

class IHSData(object):
    def __init__(self, data_path):
        product_file = data_path+"IHS/US/products.csv"
        material_file = data_path+"IHS/US/materials.csv"
        self.products = pd.read_csv(product_file,index_col=0).reset_index(drop=True)
        self.materials = pd.read_csv(material_file,index_col=0).reset_index(drop=True)

class ICISData(object):
    def __init__(self):
        self.countryCol = 'COUNTRY/TERRITORY'
        path = "C:\ICIS_data/US_allchemicals.xlsx"
        self.properties = ['Capacity','Statistic Production', 'Import','Export','Consumption']
        in_file = pd.ExcelFile(path)
        self.plants, self.prod, self.imps, self.exps, self.cons = [pd.DataFrame()]*5

        def append_data(prev,region,breaks,start):
          br_end = breaks[start+1] if start+1 < len(breaks) else -1
          return prev.append(region[breaks[start]+1:br_end].dropna(subset=[self.countryCol]), ignore_index=True)

        for sheet in in_file.sheet_names[2:]:
          region = pd.read_excel(in_file, sheet_name=sheet)
          region['Region'] = [sheet]*len(region)
          breaks = region.loc[region['PRODUCT'].isin(self.properties)].index
          self.plants, self.prod, self.imps, self.exps, self.cons = [append_data(attr,region,breaks,i) for i, attr in enumerate([self.plants, self.prod, self.imps, self.exps, self.cons])]

lca = LCA(data_path).location('United States')
ihs = IHSData(data_path)
icis = ICISData()

In [83]:
lca

,name,location,CO2e
0,"1,1-difluoroethane, HFC-152a",US,5.916414
1,"1,1-dimethylcyclopentane",RER,0.802418
2,"1,1-dimethylcyclopentane",RoW,0.870474
3,1-butanol,RoW,3.045196
4,1-pentanol,RoW,5.078633
...,...,...,...
573,xylene,RoW,1.703801
574,zinc monosulfate,RoW,0.702319
575,zinc oxide,RoW,0.815520
576,zinc sulfide,RoW,2.135332


In [90]:
icis.prod['PRODUCT'].unique()
icis.plants
#icis.prod

,PRODUCT,COUNTRY/TERRITORY,STATE,COMPANY,SITE,#,ROUTE,TECHNOLOGY,LICENSOR,START_YR,...,None.58,2037,None.59,2038,None.60,2039,None.61,2040,None.62,Region
0,2-ETHYLHEXANOL,UNITED STATES,n.a.,~ADDITIONAL,~UNKNOWN USA,1.0,BUTYRALDEHYDE,OXO,n.a.,2023,...,,220,,220,,220,,220,,CountryTerritory Summary
1,2-ETHYLHEXANOL,UNITED STATES,TEXAS,BASF CORP.,FREEPORT/TX,1.0,BUTYRALDEHYDE,OXO,BASF,-,...,,160,,160,,160,,160,,CountryTerritory Summary
2,2-ETHYLHEXANOL,UNITED STATES,TEXAS,BASF CORP.,PASADENA/TX,1.0,BUTYRALDEHYDE,OXO,UN. CARBIDE/DAVY MCKEE/J.,1983,...,None,-,None,-,None,-,None,-,None,CountryTerritory Summary
3,2-ETHYLHEXANOL,UNITED STATES,TEXAS,BASF CORP.,PASADENA/TX,2.0,BUTYRALDEHYDE,n.a.,n.a.,2017,...,,100,,100,,100,,100,,CountryTerritory Summary
4,2-ETHYLHEXANOL,UNITED STATES,TEXAS,DOW CHEMICAL,TEXAS CITY/TX,1.0,BUTYRALDEHYDE,OXO,UN. CARBIDE/DAVY MCKEE/J.,-,...,,55,,55,,55,,55,,CountryTerritory Summary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,VISBREAKING,UNITED STATES,CALIFORNIA,UNOCAL CORP.,LOS ANGELES/CA,1.0,VISBREAKING,n.a.,n.a.,-,...,None,-,None,-,None,-,None,-,None,CountryTerritory Summary
4413,VISBREAKING,UNITED STATES,TEXAS,VALERO REFINING,CORPUS CHRISTI/TX,2.0,VISBREAKING,n.a.,n.a.,1982,...,None,-,None,-,None,-,None,-,None,CountryTerritory Summary
4414,VISBREAKING,UNITED STATES,LOUISIANA,VALERO REFINING,GOOD HOPE/LA,1.0,VISBREAKING,n.a.,n.a.,1980,...,None,-,None,-,None,-,None,-,None,CountryTerritory Summary
4415,VISBREAKING,UNITED STATES,TEXAS,VALERO REFINING,HOUSTON/TX,1.0,VISBREAKING,n.a.,n.a.,1978,...,None,-,None,-,None,-,None,-,None,CountryTerritory Summary


In [88]:
ihs.materials

,Code,Data Version,Source,Target,Research Year,Geography,Plant capacity,Capacity unit,Product,Value,...,Product fixed costs,"Product Overhead + Tax, Ins.",Product Plant Cash Cost,Product Depreciation,Product Plant Gate Costs,"Product G&A, Sales, Res.",Production Cost,Product ROI (15%),Product Value,Product Price
0,1M-981,2021 Q3,CATALYST,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.068470,MM KG/yr,ETHYLENE,NaN,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
1,1M-981,2021 Q3,CAUSTIC SODA (50%),ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.068470,MM KG/yr,ETHYLENE,0.006260,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
2,1M-981,2021 Q3,"METHANOL, CRUDE",ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.068470,MM KG/yr,ETHYLENE,5.941383,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
3,1M-981,2021 Q3,C4-C5 MIXTURE,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.068470,MM KG/yr,ETHYLENE,-0.436700,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
4,1M-981,2021 Q3,ETHANE-RICH GAS,ETHYLENE BY THE UOP/HYDRO METHANOL TO OLEFINS ...,2002.0,"United States, Gulf Coast",400.068470,MM KG/yr,ETHYLENE,-0.031340,...,5.893464,6.625514,34.643831,22.118850,56.762680,2.781679,59.544360,33.178274,92.722634,100.420560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,1M-1457,2021 Q3,HYDROGEN.,"BIO-BASED 1,4-BUTANEDIOL VIA SUCCINIC ACID ROU...",2011.0,"United States, Gulf Coast",29.937096,MM KG/yr,"1,4-BUTANEDIOL",0.109000,...,27.305847,28.141565,673.899861,62.139265,736.039126,43.644633,779.683759,93.208898,872.892658,172.776275
2904,1M-1457,2021 Q3,COOLING WATER,"BIO-BASED 1,4-BUTANEDIOL VIA SUCCINIC ACID ROU...",2011.0,"United States, Gulf Coast",29.937096,MM KG/yr,"1,4-BUTANEDIOL",69.099916,...,27.305847,28.141565,673.899861,62.139265,736.039126,43.644633,779.683759,93.208898,872.892658,172.776275
2905,1M-1457,2021 Q3,ELECTRICITY,"BIO-BASED 1,4-BUTANEDIOL VIA SUCCINIC ACID ROU...",2011.0,"United States, Gulf Coast",29.937096,MM KG/yr,"1,4-BUTANEDIOL",529.109429,...,27.305847,28.141565,673.899861,62.139265,736.039126,43.644633,779.683759,93.208898,872.892658,172.776275
2906,1M-1457,2021 Q3,PROCESS WATER,"BIO-BASED 1,4-BUTANEDIOL VIA SUCCINIC ACID ROU...",2011.0,"United States, Gulf Coast",29.937096,MM KG/yr,"1,4-BUTANEDIOL",27.039098,...,27.305847,28.141565,673.899861,62.139265,736.039126,43.644633,779.683759,93.208898,872.892658,172.776275


In [89]:
merge = pd.merge(lca, icis.plants, left_on=lca['name'].str.lower(), right_on=icis.plants['PRODUCT'].str.lower(), how="left")
merge['PRODUCT'].unique()

array([nan, 'ACETALDEHYDE', 'ACRYLIC ACID', 'ACRYLONITRILE',
       'ADIPIC ACID', 'ANILINE', 'BENZENE', 'BUTADIENE', 'BUTYL ACETATE',
       'CUMENE', 'CYCLOHEXANE', 'CYCLOHEXANONE', 'EPICHLOROHYDRIN',
       'ETHYL ACETATE', 'ETHYLENE', 'ETHYLENE OXIDE', 'ISOBUTANOL',
       'ISOPROPANOL', 'MALEIC ANHYDRIDE', 'METHANOL', 'PHENOL',
       'PHTHALIC ANHYDRIDE', 'POLYBUTADIENE', 'PROPYLENE', 'STYRENE'],
      dtype=object)

In [80]:
merge_cons = pd.merge(lca, icis.cons, left_on=lca['name'].str.lower(), right_on=icis.cons['DERIVATIVE'].str.lower(), how="left")
merge_cons['DERIVATIVE'].unique()

array([nan, 'ACETALDEHYDE', 'ACETIC ANHYDRIDE', 'ACETONE CYANOHYDRIN',
       'ACRYLIC ACID', 'ACRYLONITRILE', 'ADIPIC ACID', 'ANILINE',
       'BENZENE', 'BUTYL ACETATE', 'CUMENE', 'CYCLOHEXANE',
       'CYCLOHEXANONE', 'EPICHLOROHYDRIN', 'ETHYL ACETATE', 'ETHYLENE',
       'ETHYLENE OXIDE', 'FORMALDEHYDE', 'ISOBUTANOL', 'ISOPROPANOL',
       'MALEIC ANHYDRIDE', 'PHENOL', 'PHTHALIC ANHYDRIDE',
       'POLYBUTADIENE', 'PROPYLENE', 'STYRENE'], dtype=object)

In [81]:
mergeIHS = pd.merge(merge, ihs.materials, left_on=merge['PRODUCT'].str.lower(), right_on=ihs.materials['Product'].str.lower(), how="left")
mergeIHS['Product'].unique()

array([nan, 'ACETALDEHYDE', 'ACRYLONITRILE', 'ADIPIC ACID', 'ANILINE',
       'BENZENE', 'BUTADIENE', 'ETHYLENE', 'ISOPROPANOL'], dtype=object)

In [82]:
mergeIHS

,key_0,name,location,CO2e,PRODUCT,COUNTRY/TERRITORY,STATE,COMPANY,SITE,#,...,Product fixed costs,"Product Overhead + Tax, Ins.",Product Plant Cash Cost,Product Depreciation,Product Plant Gate Costs,"Product G&A, Sales, Res.",Production Cost,Product ROI (15%),Product Value,Product Price
0,"1,1-difluoroethane, hfc-152a","1,1-difluoroethane, HFC-152a",US,5.916414,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,1-dimethylcyclopentane","1,1-dimethylcyclopentane",RER,0.802418,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,1-dimethylcyclopentane","1,1-dimethylcyclopentane",RoW,0.870474,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1-butanol,1-butanol,RoW,3.045196,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1-pentanol,1-pentanol,RoW,5.078633,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70527,xylene,xylene,RoW,1.703801,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70528,zinc monosulfate,zinc monosulfate,RoW,0.702319,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70529,zinc oxide,zinc oxide,RoW,0.815520,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70530,zinc sulfide,zinc sulfide,RoW,2.135332,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
lca

,name,location,CO2e
0,"1,1-difluoroethane, HFC-152a",US,5.916414
1,"1,1-dimethylcyclopentane",RER,0.802418
2,"1,1-dimethylcyclopentane",RoW,0.870474
3,1-butanol,RoW,3.045196
4,1-pentanol,RoW,5.078633
...,...,...,...
573,xylene,RoW,1.703801
574,zinc monosulfate,RoW,0.702319
575,zinc oxide,RoW,0.815520
576,zinc sulfide,RoW,2.135332


In [119]:
# Get ICIS plants

year = 2019
plants_clean = icis.plants.drop(columns=['#','DERIVATIVE','NOTE','@','@Sub','Region']+list(filter(lambda i: 'None' in str(i) or (type(i) is int and i!=year), icis.plants.columns)))
plants_working = plants_clean[plants_clean[year] != '-'].reset_index(drop=True)
plants_working

,PRODUCT,COUNTRY/TERRITORY,STATE,COMPANY,SITE,ROUTE,TECHNOLOGY,LICENSOR,START_YR,START_MO,COMPLEX,LATITUDE,LONGITUDE,2019
0,2-ETHYLHEXANOL,UNITED STATES,TEXAS,BASF CORP.,FREEPORT/TX,BUTYRALDEHYDE,OXO,BASF,-,na,n.a.,29.003299,-95.393879,160
1,2-ETHYLHEXANOL,UNITED STATES,TEXAS,BASF CORP.,PASADENA/TX,BUTYRALDEHYDE,n.a.,n.a.,2017,na,None,29.614724,-95.052865,100
2,2-ETHYLHEXANOL,UNITED STATES,TEXAS,DOW CHEMICAL,TEXAS CITY/TX,BUTYRALDEHYDE,OXO,UN. CARBIDE/DAVY MCKEE/J.,-,na,n.a.,29.378634,-94.943043,55
3,2-ETHYLHEXANOL,UNITED STATES,TEXAS,EASTMAN CHEMICAL,LONGVIEW/TX,BUTYRALDEHYDE,OXO,EASTMAN CHEMICAL,-,na,n.a.,32.437345,-94.702061,250
4,ABS,UNITED STATES,OHIO,INEOS STYROLUTION,ADDYSTON/OH,VIRGIN RESIN,POLYMERIZATION,MONSANTO,1972,na,None,39.135425,-84.713303,195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,VINYL ACETATE M.,UNITED STATES,TEXAS,DOW CHEMICAL,TEXAS CITY/TX,ETHYLENE,ACETOXYLATION,BAYER,1976,na,*,29.378634,-94.943043,365
2454,VINYL ACETATE M.,UNITED STATES,TEXAS,KURARAY CO.,LA PORTE/TX,ETHYLENE,ACETOXYLATION,n.a.,-,na,n.a.,29.702605,-95.038387,335
2455,VINYL ACETATE M.,UNITED STATES,TEXAS,LYONDELLBASELL INDUSTRIES,LA PORTE/TX,ETHYLENE,ACETOXYLATION,QUANTUM,1970,na,n.a.,29.708512,-95.065409,385
2456,VISBREAKING,UNITED STATES,HAWAII,PAR HAWAII REFINING,KAPOLEI/HI,VISBREAKING,n.a.,n.a.,1987,na,*,21.304607,-158.097343,650


In [174]:
# Associate with IHS process and merge
import difflib
plants_working['ihsProcess'] = list(map(lambda x: next(iter(difflib.get_close_matches(str(x), ihs.products['Process'])),None), plants_working['PRODUCT']+' '+plants_working['ROUTE']+' '+plants_working['TECHNOLOGY']))

plant_products = pd.merge(plants_working, ihs, left_on=plants_working['ihsProcess'], right_on=ihs.products['Process'], how="left")


# df2 = ihs.products.sort_values('Name')
# df2['NameNew'] = df2['Name'].apply(lambda x: next(iter(difflib.get_close_matches(x, plants_working['ROUTE'])),None))
# df2['ProcessNew'] = df2['Process'].apply(lambda x: next(iter(difflib.get_close_matches(x, plants_working['PRODUCT'])),None))
#.merge(df2)
plants_working

TypeError: Can only merge Series or DataFrame objects, a <class '__main__.IHSData'> was passed

In [ ]:
# Sparse matrix with all materials for process frmo IHS

In [ ]:
# Associate with Y vector from EcoInvent

In [145]:
import difflib
plants_working['ihsProcess'] = list(map(lambda x: next(iter(difflib.get_close_matches(x, ihs.products['Process'])),None), plants_working['ROUTE']+' '+plants_working['TECHNOLOGY']))

TypeError: 'float' object is not iterable

In [172]:
l = plants_working['PRODUCT']+' '+plants_working['ROUTE']+' '+plants_working['TECHNOLOGY']
list(map(lambda x: next(iter(difflib.get_close_matches(str(x), ihs.products['Process'])),None), plants_working['PRODUCT']+' '+plants_working['ROUTE']+' '+plants_working['TECHNOLOGY']))

[None,
 None,
 None,
 None,
 'ABS RESIN BY EMULSION POLYMERIZATION',
 'ABS RESIN BY EMULSION POLYMERIZATION',
 'ABS RESIN PRODUCTION',
 'ABS RESIN BY CONTINUOUS MASS POLYMERIZATION',
 'ACETALDEHYDE FROM ETHYLENE BY ONE-STEP OXIDATION',
 'ACETIC ACID BY LOW PRESSURE CARBONYLATION OF METHANOL',
 None,
 'AZELAIC ACID FROM OLEIC ACID',
 None,
 'ACETIC ACID BY LOW PRESSURE CARBONYLATION OF METHANOL',
 'ACETIC ACID BY LOW PRESSURE CARBONYLATION OF METHANOL',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 None,
 None,
 None,
 None,
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 None,
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 'ACROLEIN BY PROPYLENE OXIDATION',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ACRYL